In [32]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torchvision.datasets import cifar
from torch.utils.data import DataLoader
from torchvision.models import resnet50

In [24]:
class UpsampleBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(UpsampleBlock, self).__init__()
        self.upsample = nn.Sequential(
            nn.ConvTranspose2d(in_channel, out_channel, kernel_size=2, stride=2),
            nn.BatchNorm2d(out_channel),
            nn.ReLU()
        )

    def forward(self, x):
        return self.upsample(x)

In [25]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(*list(resnet50().children())[:-2])

    def forward(self, x):
        return self.encoder(x)

In [48]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            UpsampleBlock(2048, 1024),
            UpsampleBlock(1024, 512),
            UpsampleBlock(512, 256),
            UpsampleBlock(256, 64),
            UpsampleBlock(64, 3)
        )

    def forward(self, x):
        return self.decoder(x)

In [49]:
class EncoderDecoder(nn.Module):
    def __init__(self):
        super(EncoderDecoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [50]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

In [11]:
train_data = cifar.CIFAR100(root='./data', train=True, transform=transforms, download=True)
test_data = cifar.CIFAR100(root='./data', train=False, transform=transforms, download=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 169001437/169001437 [05:02<00:00, 558857.66it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [34]:
batch_size = 8
learning_rate = 0.01
num_epochs = 10

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [55]:
model = EncoderDecoder().to(device)

In [33]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [57]:
for epoch in range(num_epochs):
    train_loss, val_loss = [], []
    
    model.train()
    for image, _ in train_loader:
        image = image.to(device)
        
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, image)
        train_loss.append(loss.item())

        loss.backward()
        optimizer.step()

    model.eval()
    for image, _ in test_loader:
        image = image.to(device)
        
        outputs = model(image)
        loss = criterion(outputs, image)
        val_loss.append(loss.item())

        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch} Train Loss: {train_loss[-1] \n Val Loss: {val_loss[-1]')

SyntaxError: f-string expression part cannot include a backslash (166509508.py, line 26)

In [ ]:
print(\